In [1]:
!pip install gurobipy

   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.2 MB 217.9 kB/s eta 0:00:47
   ---------------------------------------- 0.0/10.2 MB 217.9 kB/s eta 0:00:47
   ---------------------------------------- 0.1/10.2 MB 297.7 kB/s eta 0:00:35
   ---------------------------------------- 0.1/10.2 MB 467.6 kB/s eta 0:00:22
    --------------------------------------- 0.2/10.2 MB 615.9 kB/s eta 0:00:17
   - -------------------------------------- 0.3/10.2 MB 749.3 kB/s eta 0:00:14
   - -------------------------------------- 0.3/10.2 MB 853.3 kB/s eta 0:00:12
   - -------------------------------------- 0.4/10.2 MB 930.9 kB/s eta 0:00:11
   - -------------------------------------- 0.5/10.2 MB 994.6 kB/s eta 0:00:10
   -- ------------------------------------- 0.7/10.2 MB 1.3 MB/s eta 0:00:08
  


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: C:\Users\dimak\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

In [3]:
K_p = 5
Jj = 50

q = [100, 300, 400, 200, 100]

r = [[0, 177, 0, 120, 0], [0, 85, 129, 84, 55], [27, 51, 0, 0, 0], [0, 0, 36, 0, 0], [42, 0, 226, 0, 0],
     [0, 115, 0, 0, 0], [52, 163, 172, 33, 0], [0, 0, 214, 70, 0], [42, 0, 0, 0, 46], [0, 0, 176, 107, 0],
     [0, 163, 0, 0, 0], [20, 164, 201, 48, 48], [17, 0, 0, 0, 0], [0, 166, 0, 114, 0], [0, 106, 0, 0, 0],
     [8, 0, 0, 0, 0], [0, 0, 0, 0, 55], [41, 0, 141, 0, 0], [0, 29, 118, 84, 29], [6, 0, 145, 0, 22],
     [0, 0, 32, 0, 0], [22, 165, 0, 98, 0], [24, 0, 223, 0, 0], [51, 0, 0, 111, 0], [56, 0, 142, 0, 0],
     [46, 0, 0, 0, 0], [0, 0, 0, 117, 9], [0, 0, 0, 24, 0], [0, 31, 0, 0, 40], [57, 0, 158, 0, 57],
     [35, 0, 128, 0, 0], [42, 0, 0, 0, 29], [26, 0, 0, 0, 0], [41, 66, 0, 0, 11], [57, 48, 0, 0, 0],
     [0, 149, 0, 58, 0], [23, 0, 0, 0, 0], [32, 0, 52, 0, 49], [0, 0, 0, 0, 35], [17, 85, 80, 0, 0],
     [0, 168, 0, 0, 0], [0, 147, 129, 0, 0], [39, 59, 0, 0, 0], [12, 0, 0, 75, 0], [14, 126, 0, 88, 26],
     [30, 0, 188, 0, 0], [0, 0, 0, 69, 0], [0, 0, 33, 0, 0], [15, 81, 0, 119, 0], [0, 75, 0, 0, 0]]

p = [1, 10, 2, 6, 1, 2, 9, 1, 10, 6,
     6, 5, 7, 2, 4, 2, 9, 4, 6, 8,
     1, 2, 8, 1, 5, 6, 6, 4, 2, 5,
     5, 8, 5, 7, 8, 6, 8, 4, 3, 9,
     4, 9, 2, 8, 3, 10, 6, 1, 9, 6]

a = [[0, 2], [2, 6], [4, 6], [6, 3], [3, 7],
     [7, 24], [24, 19], [17, 18], [18, 19], [20, 24],
     [25, 19], [29, 25], [19, 32], [32, 38], [36, 33],
     [34, 33], [33, 38], [38, 37], [37, 42], [42, 45],
     [45, 49], [47, 44], [46, 44], [44, 43], [43, 49]]

In [5]:
# Словарь предшественников
predecessors_dict = {}

for edge in a:
    successor = edge[1]
    predecessor = edge[0]

    if successor not in predecessors_dict:
        predecessors_dict[successor] = [predecessor]
    else:
        predecessors_dict[successor].append(predecessor)

# Список предшественников для каждой работы
P = [predecessors_dict.get(successor, []) for successor in range(Jj)]

# Список предшественников для каждой работы
P = [predecessors_dict.get(successor, []) for successor in range(Jj)]

# Множество индексов работ
J = np.arange(0, Jj, 1)

# Массив времени завершения работ
s = [-1] * Jj

In [11]:
# Функция определения завершенных работ
def C(t):
    return set(j for j in range(Jj) if s[j] >= 0 and s[j] + p[j] <= t)

# Функция определения выполняемых работ
def A(t):
    return set(j for j in range(Jj) if s[j] >= 0 and s[j] <= t and s[j] + p[j] > t)

# Функция объединения множеств
def union(a, b):
    return list(set(a + b))

# Функция поиска множества A(t) без элементов множества C(t)
def search_A(B, C):
    A = B.copy()
    for element in C:
        if element in A:
            A.remove(element)
    return A

# Функция определения работ, которые можно начать в момент времени t
def D(t):
    d = []
    union_1 = A(t).union(C(t))
    need = search_A(list(J), list(union_1))
    for j in list(need):
        pred_j = P[j]
        P_set = set(pred_j)
        C_set = set(C(t))
        if P_set.issubset(C_set):
            if len(list(A(t))) > 0:
                for i in list(A(t)):
                    for k in range(0, K_p):
                        if r[i][k] + r[j][k] <= q[k]:
                            d.append(j)
            else:
                d = need
    d = set(d)
    return d

# Функция определения множества D(t_l) и решения задачи оптимизации
def f_d(t_l):
    d = list(D(t_l))
    R = [0] * Jj
    for j in list(D(t_l)):
        summ = 0
        for k in range(0, K_p):
            summ += (float(r[j][k]) / q[k])
        R[j] = summ

    n = Jj
    m = gp.Model()
    x = m.addVars(n, vtype=GRB.BINARY)
    for k in range(0, K_p):
        s1 = 0
        s2 = gp.LinExpr()
        for i in list(A(t_l)):
            s1 += r[i][k]
        for j in d:
            s2 += x[j] * r[j][k]
        m.addConstr(s2 <= q[k] - s1)

    s3 = 0
    for j in d:
        s3 += R[j] * x[j]

    m.setParam('OutputFlag', 0)
    m.setObjective(s3, GRB.MAXIMIZE)
    m.optimize()
    ans = [j for j in d if x[j].x == 1]
    return ans

# Функция поиска минимума между a и b
def min_(a, b):
    return a if a < b else b

def find_tl(l):
    if l < 2:
        return 0
    a = list(A(t[l - 1]))
    minn = 100000
    for i in range(len(a)):
        minn = min_(minn, (s[a[i]] + p[a[i]]))
    return minn

In [12]:
# Массив временных моментов
t = [0] * 1000
l = 0
s = [-1] * Jj


In [13]:
# Основной цикл
while (len(union(list(A(t[l])), list(C(t[l])))) < len(J)):
    l += 1
    t[l] = find_tl(l)
    Q = [0] * K_p
    for k in range(0, K_p):
        s_r = 0
        for j in A(t[l]):
            s_r += r[j][k]
            Q[k] = q[k] - s_r
    D_new = f_d(t[l])
    print(D_new, l, t[l])

    for j in D_new:
        s[j] = t[l]

Restricted license - for non-production use only - expires 2025-11-24


[9, 10, 12, 14, 20, 27, 28, 29, 32, 46, 47] 1 0
[] 2 1
[] 3 2
[5] 4 4
[15, 16, 22, 31] 5 5
[26, 35, 41] 6 6
[36] 7 7
[] 8 12
[44, 23] 9 13
[4] 10 14
[1, 17, 30] 11 15
[21] 12 16
[13] 13 18
[34] 14 19
[39, 18] 15 20
[43] 16 25
[] 17 26
[8, 11] 18 27
[] 19 29
[0, 33] 20 32
[48] 21 33
[2] 22 37
[38, 6] 23 39
[37] 24 42
[42] 25 46
[3, 7, 40, 49, 19, 25] 26 48
[45] 27 49
[] 28 52
[] 29 54
[24] 30 56


In [14]:
print(C(t[l]), A(t[l]))

print("C_max =", max([s[i] + p[i] for i in A(t[l])]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49} {24, 45}
C_max = 61
